# 1. Environment Setup

## 1.1 Packages


Reference:

https://github.com/pyy0715/Neural-Collaborative-Filtering/blob/master/src/model.py

In [ ]:
! pip install tensorboardX
! pip install comet_ml --quiet
! pip install tensorboardX --quiet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# PyTorch packages
import torch
# Data Processing and Visualization
import pandas as pd
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.sparse.linalg import svds
from mpl_toolkits import mplot3d

# Sklearn Packages
from sklearn.metrics import mean_squared_error

# Python package
import re

# IO Packages
import os

import comet_ml
comet_ml.init(project_name='NCF_adversarial_baseline')

import torchvision.utils as vutils
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import datetime

COMET INFO: Comet API key is valid


### Key: mJW6twxmHY1SOLeltflyLMbaH
### Key: LKMVBJt1vZVGiUCt8cwgck50H

## 1.2 Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.3 Visualize in tensorboard

In [ ]:
# Helper function to display logged assets in the Comet UI
def display(tab=None):
  experiment = comet_ml.get_global_experiment()
  experiment.display(tab=tab)

# 2. NCF model

## config.py

In [ ]:
# DATA_URL = "http://files.grouplens.org/datasets/movielens/ml-100k/u.data"
# MAIN_PATH = '/content/Neural-Collaborative-Filtering/src/Neural-Collaborative-Filtering'
# DATA_PATH = MAIN_PATH + 'data/ml-1m/ratings.dat'
MAIN_PATH = "/content/drive/MyDrive/Recommender System Codes/"
DATA_PATH="/content/drive/MyDrive/Recommender System Codes/MovieLen1M/"
MODEL_PATH = MAIN_PATH + 'models/'
MODEL = 'ml-1m_Neu_MF'

## data_utils.py

In [ ]:
import random
import numpy as np 
import pandas as pd 
import torch
# import config 

class NCF_Data(object):
	"""
	Construct Dataset for NCF
	"""
	def __init__(self, args, ratings):
		self.ratings = ratings
		self.num_ng = args.num_ng
		self.num_ng_test = args.num_ng_test
		self.batch_size = args.batch_size
# reindex 
		self.preprocess_ratings = self._reindex(self.ratings)
# all unique users and items in ratings.csv
		self.user_pool = set(self.ratings['user_id'].unique())
		self.item_pool = set(self.ratings['item_id'].unique())
# split into train and test set, we should add a validation set
		self.train_ratings, self.test_ratings = self._leave_one_out(self.preprocess_ratings)
# interacted items (positive samples?)
		self.negatives = self._negative_sampling(self.preprocess_ratings)
		random.seed(args.seed)
	
	def _reindex(self, ratings):
		"""
		Process dataset to reindex userID and itemID, also set rating as binary feedback
		"""
		user_list = list(ratings['user_id'].drop_duplicates())
		user2id = {w: i for i, w in enumerate(user_list)}

		item_list = list(ratings['item_id'].drop_duplicates())
		item2id = {w: i for i, w in enumerate(item_list)}

		ratings['user_id'] = ratings['user_id'].apply(lambda x: user2id[x])
		ratings['item_id'] = ratings['item_id'].apply(lambda x: item2id[x])
		ratings['rating'] = ratings['rating'].apply(lambda x: float(x > 0))
		return ratings

	def _leave_one_out(self, ratings):
		"""
		leave-one-out evaluation protocol in paper https://www.comp.nus.edu.sg/~xiangnan/papers/ncf.pdf
		"""
		ratings['rank_latest'] = ratings.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)
		test = ratings.loc[ratings['rank_latest'] == 1]
		train = ratings.loc[ratings['rank_latest'] > 1]
		assert train['user_id'].nunique()==test['user_id'].nunique(), 'Not Match Train User with Test User'
		return train[['user_id', 'item_id', 'rating']], test[['user_id', 'item_id', 'rating']]
	

	def _negative_sampling(self, ratings):
		# After grroupby, it becomes "UserID, interacted items with THIS user"
		interact_status = (
			ratings.groupby('user_id')['item_id']
			.apply(set)
			.reset_index()
			.rename(columns={'item_id': 'interacted_items'}))
	# negative_items refers to items that have no interaction with THIS user_id
		interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: self.item_pool - x)
	# negative_items refers to a random subset of negatives_items with THIS user
		interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, self.num_ng_test))
		return interact_status[['user_id', 'negative_items', 'negative_samples']]

	def get_train_instance(self):
		users, items, ratings = [], [], []
		# train_ratings is a n * m matrix, where 1 indicating interaction and 0 indicating negative items for each user 
		train_ratings = pd.merge(self.train_ratings, self.negatives[['user_id', 'negative_items']], on='user_id')
		# train_ratings['negatives'] is a subset of negative items for each user 
		train_ratings['negatives'] = train_ratings['negative_items'].apply(lambda x: random.sample(x, self.num_ng))
		for row in train_ratings.itertuples():
			users.append(int(row.user_id))
			items.append(int(row.item_id))
			ratings.append(float(row.rating)) 
			for i in range(self.num_ng):
				users.append(int(row.user_id))
				items.append(int(row.negatives[i]))
				ratings.append(float(0))  # negative samples get 0 rating
		dataset = Rating_Datset(
			user_list=users,
			item_list=items,
			rating_list=ratings)
		# return a dataloader with full interacted items and self.num_ng number of negative items with rating = 0
		return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=4)

	# same with get_train_instance
	def get_test_instance(self):
		users, items, ratings = [], [], []
		test_ratings = pd.merge(self.test_ratings, self.negatives[['user_id', 'negative_samples']], on='user_id')
		for row in test_ratings.itertuples():
			users.append(int(row.user_id))
			items.append(int(row.item_id))
			ratings.append(float(row.rating))
			# print('positive sample')
			# default 100 ng_items with 1 pos items at index 0
			for i in getattr(row, 'negative_samples'):
				users.append(int(row.user_id))
				items.append(int(i))
				ratings.append(float(0))
				# print('negative sample')
		dataset = Rating_Datset(
			user_list=users,
			item_list=items,
			rating_list=ratings)
		return torch.utils.data.DataLoader(dataset, batch_size=self.num_ng_test+1, shuffle=False, num_workers=4)


class Rating_Datset(torch.utils.data.Dataset):
	def __init__(self, user_list, item_list, rating_list):
		super(Rating_Datset, self).__init__()
		self.user_list = user_list
		self.item_list = item_list
		self.rating_list = rating_list

	def __len__(self):
		return len(self.user_list)

	def __getitem__(self, idx):
		user = self.user_list[idx]
		item = self.item_list[idx]
		rating = self.rating_list[idx]
		
		return (
			torch.tensor(user, dtype=torch.long),
			torch.tensor(item, dtype=torch.long),
			torch.tensor(rating, dtype=torch.float)
			)

In [ ]:
movies = pd.read_csv(
	DATA_PATH+"movies.dat", 
	sep="::", 
	names = ['item_id', 'title', 'genre'], 
	engine='python',
	encoding = "ISO-8859-1")
print(movies)

      item_id                               title  \
0           1                    Toy Story (1995)   
1           2                      Jumanji (1995)   
2           3             Grumpier Old Men (1995)   
3           4            Waiting to Exhale (1995)   
4           5  Father of the Bride Part II (1995)   
...       ...                                 ...   
3878     3948             Meet the Parents (2000)   
3879     3949          Requiem for a Dream (2000)   
3880     3950                    Tigerland (2000)   
3881     3951             Two Family House (2000)   
3882     3952               Contender, The (2000)   

                             genre  
0      Animation|Children's|Comedy  
1     Adventure|Children's|Fantasy  
2                   Comedy|Romance  
3                     Comedy|Drama  
4                           Comedy  
...                            ...  
3878                        Comedy  
3879                         Drama  
3880                         D

In [ ]:
movies.loc[movies['item_id'] == 3952, 'genre'].tolist()[0]

'Drama|Thriller'

## evaluate.py

In [ ]:
import numpy as np
import torch
import collections

def hit(pos_item, pred_items):
  if pos_item in pred_items:
    return 1
  return 0


def ndcg(pos_item, pred_items):
  if pos_item in pred_items:
    index = pred_items.index(pos_item)
    return np.reciprocal(np.log2(index+2))
  return 0

# Gini index to measure diversity of the recommended list
def gini(pred_items):
  gini_index = 1
  k = len(pred_items)
  genres = []
  # genres is a list of lists
  for item in pred_items:
    genre = movies.loc[movies['item_id'] == item, 'genre'].tolist()
    genres.append(genre)
  # genres_lst is a list
  genres_lst = [item for sublist in genres for item in sublist]
  counter = collections.Counter(genres_lst)
  frequency_lst = [counter[x]/k for x in sorted(counter.keys())]
  for frequency in frequency_lst:
    gini_index -= frequency**2
  return gini_index

# Entropy index to measure diversity of the recommended list
def entropy(pred_items):
  entropy_index = 0
  k = len(pred_items)
  genres = []
  for item in pred_items:
    genre = movies.loc[movies['item_id'] == item, 'genre'].tolist()
    genres.append(genre)
  genres_lst = [item for sublist in genres for item in sublist]
  counter = collections.Counter(genres_lst)
  frequency_lst = [counter[x]/k for x in sorted(counter.keys())]
  for frequency in frequency_lst:
    entropy_index = entropy_index + frequency * math.log(frequency,math.e)
  return -entropy_index

'''
def coverage(pred_items):
  k = len(pred_items)
  item_len= len(items)
  return k/item_len

def personalization():
  chech if the system recommends similiar items to all uers. Can we measure this if the available items are different for each user? 
'''




# def metrics(model, test_loader, top_k, device):
#   HR, NDCG, GINI, Entropy  = [], [], [], []
#   itemMetricCollection = torch.empty(len(test_loader),top_k)
#   for i, (user, item, label) in enumerate(test_loader):
#     user = user.to(device)
#     item = item.to(device)
#     #print('user:',user.shape,user)
#     #print('item:',item.shape,item)

#     predictions = model(user, item)
#     _, indices = torch.topk(predictions, top_k)
#     recommends = torch.take(
#         item, indices).cpu().numpy().tolist()
#     pos_item = item[0].item()
#     #print('pos_item:',pos_item,'recommends:',recommends)
#     # print(a) # leave one-out evaluation has only one item per user
#     HR.append(hit(pos_item, recommends))
#     NDCG.append(ndcg(pos_item, recommends))
#     GINI.append(gini(recommends))
#     Entropy.append(entropy(recommends))
#   return np.mean(HR), np.mean(NDCG), np.mean(GINI), np.mean(Entropy)

def itemMetric(recommendedCollection, average, top_k, diversity=False):
  # TODO: popular vs long-tail items
  """
  pred_items: 6040*10 list of the recommended items.
  """
  # Figure out the genres 
  from collections import Counter
  import numpy as np

  recommendationDict = dict(Counter(recommendedCollection.flatten().cpu().numpy().tolist()))

  itemAppearanceDict = dict(sorted(recommendationDict.items(), key = lambda kv:(-kv[1], kv[0])))

  itemAppearanceList = np.array(list(recommendationDict.values()))

  freq = itemAppearanceList/itemAppearanceList.sum()

  # print(freq)

  # Figure out the genres 
  if diversity:
    return 
  # Do the statistics(个数)
  else:
    return sum(freq<average)/(6040*top_k)

def metrics(model, test_loader, top_k, average, device):

  HR, NDCG, GINI, Entropy, longTail, adv_weights_for_print  = [], [], [] ,[], [], None

  itemMetricCollection = torch.empty(len(test_loader),top_k)

  # Get the weights
  # if adv_weights_for_print is not None:
  #   print("adv_weights",adv_weights_for_print)
  #   print("weight_size",adv_weights_for_print.size())

  for i, (user, item, label) in enumerate(test_loader):
    user = user.to(device)
    item = item.to(device)

    # item_indices, position information
    predictions = model(user, item)
    # print('predictions length',predictions.shape)
    _, indices = torch.topk(predictions, top_k)

    # Actual item_id, by taking out the indices
    recommends = torch.take(item, indices).cpu().numpy().tolist()

    # For itemMetric
    itemMetricCollection[i]=torch.tensor(recommends)

    # For the hit and NDCG
    pos_item = item[0].item()

    # User Centric Metrics
    HR.append(hit(pos_item, recommends))
    NDCG.append(ndcg(pos_item, recommends))
    # GINI.append(gini(recommends))
    # Entropy.append(entropy(recommends))
    longTail.append(itemMetric(itemMetricCollection,average,top_k))
    # print('recommends length',len(recommends),'longTail:',longTail)

  # Handling item metrics
  return np.mean(HR), np.mean(NDCG), np.mean(GINI), np.mean(Entropy), np.mean(longTail)

##model.py

In [ ]:
import torch
import torch.nn as nn

class Generalized_Matrix_Factorization(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(Generalized_Matrix_Factorization, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args.factor_num

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.affine_output = nn.Linear(in_features=self.factor_num, out_features=1)
        # self.logistic = nn.Sigmoid()
        self.logistic = nn.ReLU()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        # print('item_embedding',item_embedding)
        # ELEMENT-WISE PRODUCT IN GMF
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass

class Multi_Layer_Perceptron(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(Multi_Layer_Perceptron, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num = args.factor_num
        self.layers = args.layers

        self.embedding_user = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num)
        self.embedding_item = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layers[:-1], self.layers[1:])):
            self.fc_layers.append(nn.Linear(in_size, out_size))

        self.affine_output = nn.Linear(in_features=self.layers[-1], out_features=1)
        self.logistic = nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        # print('user_embedding',user_embedding)
        item_embedding = self.embedding_item(item_indices)
        # print('item_embedding',item_embedding)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = nn.ReLU()(vector)
            # vector = nn.BatchNorm1d()(vector)
            # vector = nn.Dropout(p=0.5)(vector)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass



class NeuMF(nn.Module):
    def __init__(self, args, num_users, num_items):
        super(NeuMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.factor_num_mf = args.factor_num
        self.factor_num_mlp =  int(args.layers[0]/2)
        self.layers = args.layers
        self.dropout = args.dropout

        self.embedding_user_mlp = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mlp)
        self.embedding_item_mlp = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mlp)

        self.embedding_user_mf = nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.factor_num_mf)
        self.embedding_item_mf = nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.factor_num_mf)

        self.fc_layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(args.layers[:-1], args.layers[1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))
            self.fc_layers.append(nn.ReLU())

        self.affine_output = nn.Linear(in_features=args.layers[-1] + self.factor_num_mf, out_features=1)
        self.logistic = nn.Sigmoid()
        self.init_weight()

    def init_weight(self):
        nn.init.normal_(self.embedding_user_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mlp.weight, std=0.01)
        nn.init.normal_(self.embedding_user_mf.weight, std=0.01)
        nn.init.normal_(self.embedding_item_mf.weight, std=0.01)
        
        for m in self.fc_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                
        nn.init.xavier_uniform_(self.affine_output.weight)

        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, user_indices, item_indices):
        user_embedding_mlp = self.embedding_user_mlp(user_indices)
        item_embedding_mlp = self.embedding_item_mlp(item_indices)
        # print('item_embedding_mlp',item_embedding_mlp,item_embedding_mlp.shape)

        user_embedding_mf = self.embedding_user_mf(user_indices)
        item_embedding_mf = self.embedding_item_mf(item_indices)
        # print('item_embedding_mf',item_embedding_mf,item_embedding_mf.shape)

        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)  # the concat latent vector
        mf_vector =torch.mul(user_embedding_mf, item_embedding_mf)

        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.fc_layers[idx](mlp_vector)

        vector = torch.cat([mlp_vector, mf_vector], dim=-1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating.squeeze()

## util.py

In [ ]:
import os
import random
import numpy as np 
import torch

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

## main.py

In [ ]:
import os
import time
import argparse
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tensorboardX import SummaryWriter

# import model 
# import config 
# import util
# import data_utils
# import evaluate


parser = argparse.ArgumentParser()
parser.add_argument("--seed", 
	type=int, 
	default=42, 
	help="Seed")
parser.add_argument("--lr", 
	type=float, 
	default=0.001, 
	help="learning rate")
parser.add_argument("--dropout", 
	type=float,
	default=0.2,  
	help="dropout rate")
parser.add_argument("--batch_size", 
	type=int, 
	default=256, 
	help="batch size for training")
parser.add_argument("--epochs", 
	type=int,
	default=15,  
	help="training epoches")
parser.add_argument("--top_k", 
	type=int, 
	default=10, 
	help="compute metrics@top_k")
parser.add_argument("--factor_num", 
	type=int,
	default=32, 
	help="predictive factors numbers in the model")
parser.add_argument("--layers",
    nargs='+', 
    default=[64,32,16,8],
		# default=[16,8],
    help="MLP layers. Note that the first layer is the concatenation of user and item embeddings. So layers[0]/2 is the embedding size.")
parser.add_argument("--num_ng", 
	type=int,
	default=4, 
	help="Number of negative samples for training set")
# default: 100 negative items with 1 positive items
parser.add_argument("--num_ng_test", 
	type=int,
	default=100, 
	help="Number of negative samples for test set")
parser.add_argument("--out", 
	default=True,
	help="save model or not")

# set device and parameters
# args = parser.parse_args() 
# Use this line for jupyter notebook
args, unknown = parser.parse_known_args()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter(comet_config={"disabled": False})

# seed for Reproducibility
# util.seed_everything(args.seed)
#seed_everything(args.seed)

# load data
# user_data = pd.read_csv(DATA_PATH+"users.dat", sep = "::", header=None, names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
# movie_data = pd.read_csv(DATA_PATH+"movies.dat", sep = "::", header=None, encoding ="latin-1", names=['MovieID', 'Title', 'Genres'])
# rating_data = pd.read_csv(DATA_PATH+"ratings.dat", sep = "::", header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])


ml_1m = pd.read_csv(
	DATA_PATH+"ratings.dat", 
	sep="::", 
	names = ['user_id', 'item_id', 'rating', 'timestamp'], 
	engine='python')

movies = pd.read_csv(
	DATA_PATH+"movies.dat", 
	sep="::", 
	names = ['item_id', 'title', 'genre'], 
	engine='python',
	encoding = "ISO-8859-1")

# set the num_users, items
num_users = ml_1m['user_id'].nunique()+1
num_items = ml_1m['item_id'].nunique()+1

# construct the train and test datasets
data = NCF_Data(args, ml_1m)

train_loader =data.get_train_instance()
test_loader =data.get_test_instance()

# set model and loss, optimizer
model = NeuMF(args, num_users, num_items)
model = model.to(device)
loss_function = nn.BCELoss()
# loss_function = nn.HingeEmbeddingLoss()
average= num_items / len(ml_1m)
optimizer = optim.SGD(model.parameters(), lr=args.lr)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)

# train, evaluation
best_hr = 0
for epoch in range(1, args.epochs+1):
	model.train() # Enable dropout (if have).
	start_time = time.time()

	for user, item, label in train_loader:
		user = user.to(device)
		item = item.to(device)
		# print('item,'item)
		label = label.to(device)

		optimizer.zero_grad()
		prediction = model(user, item)
		loss_primary = loss_function(prediction, label) 
		# print('prediction',prediction,'loss_primary',loss_primary)
		# loss_primary *= w.detach()
		loss_primary.backward()
		# optimize based on primary loss 
		optimizer.step()
		writer.add_scalar('loss_primary', loss_primary.item(), epoch)
	
	model.eval()
	# itemAppearanceDict = metricsItem(model, test_loader, args.top_k, device)
	# print(itemAppreanceDict)

	HR, NDCG, GINI, Entropy, Longtail  = metrics(model, test_loader, args.top_k, average, device)
	# writer.add_scalar('Long-tail proportion', itemAppearanceDict, epoch)
	writer.add_scalar('Perfomance/HR@10', HR, epoch)
	writer.add_scalar('Perfomance/NDCG@10', NDCG, epoch)
	writer.add_scalar('Perfomance/GINI@10', GINI, epoch)
	writer.add_scalar('Perfomance/Entropy@10', Entropy, epoch)
	writer.add_scalar('Perfomance/LongtailProportion', Longtail, epoch)
	# itemAppearanceDict = metricsItem(model, test_loader, args.top_k, device)
	# print('itemAppearanceDict',itemAppearanceDict)

	elapsed_time = time.time() - start_time
	print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
			time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
	#print("HR: {:.3f}\t NDCG: {:.3f} GINI: {:.3f}  Entropy: {:.3f} Longtail: {:.3f} ".format(np.mean(HR), np.mean(NDCG), np.mean(GINI), np.mean(Entropy),np.mean(Longtail)))
	print("HR: {:.3f}\t NDCG: {:.3f} Longtail: {:.3f} ".format(np.mean(HR), np.mean(NDCG),np.mean(Longtail)))
	if HR > best_hr:
		best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
		if args.out:
			if not os.path.exists(MODEL_PATH):
				os.mkdir(MODEL_PATH)
			torch.save(model, 
				'{}{}.pth'.format(MODEL_PATH, MODEL))

writer.close()
print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(
									best_epoch, best_hr, best_ndcg))

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
COMET INFO: Comet API key is valid
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: sklearn, torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/richard

The time elapse of epoch 001 is: 00: 10: 46
HR: 0.089	 NDCG: 0.038 Longtail: 0.247 


The time elapse of epoch 002 is: 00: 06: 12
HR: 0.097	 NDCG: 0.042 Longtail: 0.022 


The time elapse of epoch 003 is: 00: 06: 10
HR: 0.110	 NDCG: 0.050 Longtail: 0.023 


The time elapse of epoch 004 is: 00: 06: 07
HR: 0.130	 NDCG: 0.060 Longtail: 0.022 


The time elapse of epoch 005 is: 00: 06: 07
HR: 0.150	 NDCG: 0.071 Longtail: 0.020 


The time elapse of epoch 006 is: 00: 06: 02
HR: 0.176	 NDCG: 0.085 Longtail: 0.019 


The time elapse of epoch 007 is: 00: 05: 56
HR: 0.201	 NDCG: 0.096 Longtail: 0.017 


The time elapse of epoch 008 is: 00: 05: 51
HR: 0.230	 NDCG: 0.117 Longtail: 0.016 
